In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
import geopy.distance
import numpy as np
from pathlib import Path
from tqdm import tqdm


In [2]:
# BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\dev_TIR_group2\Taipei-transit-data_hub\airflow\dags\harry_GCS_BigQuery_write_cred.json"
BIGQUERY_CREDENTIALS_FILE_PATH = r"C:\dev_TIR101\Taipei-transit-data_hub\airflow\dags\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH
BQ_CLIENT = bigquery.Client()

In [3]:
def query_bq_to_df(client: bigquery.Client,sql_query:str) -> pd.DataFrame:
    try:
        query_job = client.query(sql_query)
        return query_job.to_dataframe()  # Convert result to DataFrame
    except Exception as e:
        raise Exception(f"Failed to query bigquery table, reason: {e}")

In [4]:
sql_query_realtime ="""  
    SELECT * FROM `ANDY_ETL_FACT.FACT_bike_realtime`
"""
df = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_realtime)

In [5]:
df.to_csv("bike_realtime0416-0519.csv",index=False,encoding="utf-8-sig")

In [6]:
sql_query_bike_station ="""  
    SELECT * FROM `ANDY_ETL_DIM.DIM_bike_station`
"""
df_bike_station = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_bike_station)
df_bike_station.to_csv("bike_station.csv",index=False,encoding="utf-8-sig")

In [8]:
sql_query_youbike_mrt_distance ="""  
    SELECT * FROM `ANDY_ETL_DIM.DIM_youbike_mrt_distance`
"""
df_youbike_mrt_distance = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_youbike_mrt_distance)

Exception: Failed to query bigquery table, reason: 404 Not found: Table tir101-group2-422603:ANDY_ETL_DIM.DIM_youbike_mrt_distance was not found in location asia-east1; reason: notFound, message: Not found: Table tir101-group2-422603:ANDY_ETL_DIM.DIM_youbike_mrt_distance was not found in location asia-east1

Location: asia-east1
Job ID: ecf98e50-6dfa-450c-83db-3a50e144f993


In [9]:
sql_query_mrt_station_info ="""  
    SELECT * FROM `MRT_GCS_to_BQ_SRC_ODS_DIM.DIM_MRT_static_data`
"""
df_mrt_station_info = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_mrt_station_info)

In [10]:
df_mrt_station_info.to_csv("mrt_station_info.csv",index=False,encoding="utf-8-sig")

In [10]:
sql_query_youbike_mrt_distance_groupby ="""  
    SELECT * FROM
        (SELECT  
            bike_station_id,
            mrt_station_id,
            distance,
            ROW_NUMBER() OVER (PARTITION BY bike_station_id ORDER BY distance ) AS dis_rank 
        FROM `ANDY_ETL_DIM.DIM_youbike_mrt_distance`) AS t
    WHERE t.dis_rank<=3
    ORDER BY t.bike_station_id,t.dis_rank;
"""
df_youbike_mrt_distance_groupby = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_youbike_mrt_distance_groupby)

In [20]:
sql_query_youbike_mrt_distance_groupby ="""  
    SELECT * FROM
        (SELECT  
            bike_station_id,
            mrt_station_id,
            distance,
            ROW_NUMBER() OVER (PARTITION BY bike_station_id ORDER BY distance ) AS dis_rank 
        FROM `ANDY_ETL_DIM.DIM_youbike_mrt_distance`) AS t
    WHERE t.distance<=0.5
    ORDER BY t.bike_station_id,t.dis_rank;
"""
df_youbike_mrt_distance_groupby = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_youbike_mrt_distance_groupby)

In [ ]:
df_youbike_mrt_distance_groupby

In [29]:
df_youbike_mrt_distance_groupby.groupby(["bike_station_id"])["dis_rank"].count()

bike_station_id
500101001    1
500101002    1
500101003    1
500101004    1
500101006    1
            ..
500119074    1
500119077    1
500119079    1
500119080    1
500119082    1
Name: dis_rank, Length: 726, dtype: Int64

In [11]:
sql_query_mrt_history ="""  
SELECT t_enter.date,t_enter.hour,t_enter.mrt_station,t_enter.enter_num,t_exit.exit_num
FROM
    (SELECT
        date,
        hour,
        mrt_station_name_enter AS mrt_station,
        SUM(visitors_num) AS enter_num
    FROM 
        `MRT_history.ODS_MRT_history_usage`
    WHERE DATE(date) >= '2024-03-01'
    GROUP BY date,hour,mrt_station_name_enter) AS t_enter
FULL OUTER JOIN
    (SELECT
        date,
        hour,
        mrt_station_name_exit AS mrt_station,
        SUM(visitors_num) AS exit_num
    FROM 
        `MRT_history.ODS_MRT_history_usage`
    WHERE DATE(date) >= '2024-03-01'
    GROUP BY date,hour,mrt_station_name_exit) AS t_exit
ON t_enter.date = t_exit.date AND t_enter.hour = t_exit.hour AND t_enter.mrt_station = t_exit.mrt_station
ORDER BY t_enter.date,t_enter.hour,t_enter.mrt_station DESC;
"""
df_mrt_his_groupby = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_mrt_history)

In [12]:
df_mrt_his_groupby.to_csv("mrt_history_groupby03-04.csv",encoding="utf-8-sig",index=False)

In [12]:
sql_query_time_table = """  
SELECT * 
FROM `ANDY_ETL_DIM.DIM_time_table` 
WHERE `year` BETWEEN 2023 AND 2024
ORDER BY `date`
"""
df_time_table = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_time_table)

In [14]:
df_time_table.to_csv("time_table.csv",encoding="utf-8-sig",index=False)

In [39]:
sql_query_youbike_mrt_groupby ="""  
SELECT
    bike_station_id,
    mrt_station_id,
    distance,
    ROW_NUMBER() OVER (PARTITION BY bike_station_id ORDER BY distance) AS dis_rank
FROM
    `ETL_FACT.FACT_youbike_mrt_distance`
QUALIFY
    dis_rank <= 3
ORDER BY 
    bike_station_id,dis_rank;
"""
df_youbike_mrt_groupby = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_youbike_mrt_groupby)

In [6]:
sql_query_youbike_bus ="""  
SELECT
    bike_station_id,
    bus_station_id,
    distance,
FROM
    `ETL_DIM.DIM_youbike_bus_distance`
"""
df_youbike_bus_distance =  query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_youbike_bus)

In [7]:
df_youbike_bus_distance.to_csv("youbike_bus_distance.csv",index=False,encoding="utf-8")

In [14]:
sql_query_time_table ="""  
SELECT * FROM `ETL_DIM.DIM_time_table`
"""
df_time_table =  query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query_time_table)
df_time_table.to_csv("time_table.csv",index=False,encoding="utf-8-sig")

,bike_station_id,mrt_station_id,distance,dis_rank
0,500101001,BR08,0.015941,1
1,500101001,R05,0.813848,2
2,500101001,BR09,0.813848,3
3,500101002,BR08,0.053192,1
4,500101002,R05,0.857233,2
...,...,...,...,...
4240,500119090,G08,0.785118,2
4241,500119090,BR08,0.942211,3
4242,500119091,BR08,0.890423,1
4243,500119091,BR07,1.048145,2


In [43]:
df_youbike_mrt_groupby

,bike_station_id,mrt_station_id,distance,dis_rank
0,500101001,BR08,0.015941,1
1,500101001,R05,0.813848,2
2,500101001,BR09,0.813848,3
3,500101002,BR08,0.053192,1
4,500101002,R05,0.857233,2
...,...,...,...,...
4240,500119090,G08,0.785118,2
4241,500119090,BR08,0.942211,3
4242,500119091,BR08,0.890423,1
4243,500119091,BR07,1.048145,2


In [17]:
df_youbike_mrt_distance

,bike_station_id,mrt_station_id,distance
0,500107008,O02,11.193488
1,500107008,BR15,0.093802
2,500107008,BR12,2.800712
3,500107008,BR14,1.001567
4,500107008,R10,5.664833
...,...,...,...
171210,500101119,R26,13.212745
171211,500101119,BL10,2.436063
171212,500101119,O53,7.738212
171213,500101119,O52,6.889340
